In [21]:
import numpy as np
import numpy.typing as npt


In [22]:
def sigmoid(z: npt.NDArray) -> npt.NDArray:
    return 1 / (1 + np.exp(-z))


def sigmoid_derivative(z: npt.NDArray) -> npt.NDArray:
    sig = sigmoid(z)
    return sig * (1 - sigmoid(z))


def softmax(z: npt.NDArray) -> npt.NDArray:
    z = np.array(z)
    exp_z = np.exp(z - np.max(z))
    return exp_z / np.sum(exp_z)


def relu(z: npt.NDArray) -> npt.NDArray:
    return np.maximum(0, z)


def relu_derivative(z: npt.NDArray) -> npt.NDArray:
    return (z > 0).astype(float)

In [ ]:
def cross_entropy(y_pred: npt.NDArray, y_target: npt.NDArray) -> float:
    return -np.sum(y_target * np.log(y_pred + 1e-15))

In [23]:
class Layer:
    def __init__(self, input_dim: int, units: int, activation, activation_deriv):
        self.activation = activation
        self.activation_deriv = activation_deriv
        self.W = np.random.rand(units, input_dim) * 0.1
        self.b = np.zeros((units, 1))

    def forward(self, x: npt.NDArray) -> npt.NDArray:
        self.x = x
        self.z = self.W @ x + self.b
        self.a = self.activation(self.z)

        return self.a

    def backward(self, grad_output: npt.NDArray, learning_rate: float) -> npt.NDArray:
        dz = grad_output * self.activation_deriv(self.z)


In [24]:
class NeuralNetwork:
    def __init__(self, layers: list[Layer]):
        self.layers = layers

    def forward(self, x: npt.NDArray) -> npt.NDArray:
        for layer in self.layers:
            x = layer.forward(x)

        return x

    def backward(self, grad_output: npt.NDArray, learning_rate: float) -> None:
        for layer in reversed(self.layers):
            grad_output = layer.backward(grad_output, learning_rate)

    def train(self, train_x: npt.NDArray, train_y: npt.NDArray, epochs: int, learning_rate: float) -> None:
        for epoch in range(epochs):
            total_loss: float = 0.

            for x, y in zip(train_x, train_y):
                # Reshaping to column vector.
                x: npt.NDArray = x.reshape(-1, 1)
                y: npt.NDArray = y.reshape(-1, 1)
                # Forward pass
                output = self.forward(x)
                # Loss
                loss = cross_entropy(output, y)
                total_loss += loss
                # Backpropagation
                grad_output = output - y
                self.backward(grad_output, learning_rate)

            print(f"Epoch {epoch}: loss = {total_loss}")

    def predict(self):
        ...